# Basics

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Configuration dictionary for device and learning rate
CONFIG = {
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',             # Use CUDA if available for accelerated training
    'learning_rate': 1e-4
}

# Model Definitions

In [ ]:
# Architecture 1: A Simple MLP model 
class BaselineMLP(nn.Module):
    '''
    A baseline Multi-Layer Perceptron (MLP) model.
    
    Architecture:
    - Flatten input (224x224x3).
    - 3x Fully Connected layers (Hidden: 512, 128).
    - ReLU activation throughout.
    '''
    def __init__(self, num_classes=8):
        super(BaselineMLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(224*224*3, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Architecture 2: A Simple CNN Model 
class BaselineCNN(nn.Module):
    '''
    A baseline Convolutional Neural Network (CNN) model 
    
    Architecture:
    - 2 Conv2d layers followed by 2 Fully Connected layers
    - Relu activation function 
    - Max Pooling
    '''
    def __init__(self, num_classes=8):
        super(BaselineCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Architecture 3: A ResNet-18 model From Scratch 
# This architecture is chosen for its ability to learn deep features without vanishing gradients.
class BasicBlock(nn.Module):
    '''
    A basic architecture block for a 2-D Convolutional ResNet 18 model
    
    Architecture:
    - Two convolutional layers with a shortcut connection
    - Relu activation function 
    - Batch Normalization
    '''
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
# Architecture 4: A Custom Complete ResNet Model
class CustomResNet(nn.Module):
    '''
    A completely custom ResNet model built using a 4 layer architecture block
    
    Architecture:
    - A 64-128-256-512 filter size progression
    - 4 layers of BasicBlocks
    - Relu activation function
    - Batch Normalization
    '''
    def __init__(self, block, num_blocks, num_classes=8):
        super(CustomResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():
    """Returns a ResNet18 model"""
    return CustomResNet(BasicBlock, [2, 2, 2, 2])

# Training Class

In [ ]:
# In order to reduce the amount of repeated code for training and evaluation, have created 
# a Trainer class that encapsulates all the necessary methods.
class Trainer:
    '''
    Used for all models: BaselineMLP, BaselineCNN, ResNet18, CustomResNet
    
    Details:
    - Cross Entropy Loss
    - Adam Optimizer with Weight Decay 
    - ReduceLROnPlateau Scheduler
    '''
    def __init__(self, model, model_name="model"):
        self.model = model.to(CONFIG['device'])
        self.model_name = model_name
        self.criterion = nn.CrossEntropyLoss()

        # Adam optimizer with Weight Decay for regularization 
        self.optimizer = optim.Adam(self.model.parameters(), lr=CONFIG['learning_rate'], weight_decay=1e-4)

        # Scheduler to reduce LR when validation loss plateaus 
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='min', factor=0.1, patience=3)
         
        self.history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

    def train_epoch(self, loader):
        self.model.train()
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(CONFIG['device']), labels.to(CONFIG['device'])
            
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()
            
            running_loss += loss.item() * images.size(0)
        
        return running_loss / len(loader.dataset)

    def validate(self, loader):
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in loader:
                images, labels = images.to(CONFIG['device']), labels.to(CONFIG['device'])
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                running_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
        return running_loss / len(loader.dataset), 100 * correct / total

    def fit(self, train_loader, val_loader, epochs=10):
        print(f"Start training for: {self.model_name}")
        best_acc = 0.0
        
        for epoch in range(epochs):
            train_loss = self.train_epoch(train_loader)
            val_loss, val_acc = self.validate(val_loader)
            
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            
            # Step the scheduler
            self.scheduler.step(val_loss)
            
            print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
            
            # Saving the best model
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(self.model.state_dict(), f"{self.model_name}_best.pth")
        
        print(f"Best Validation Accuracy: {best_acc:.2f}%")
        return self.history